<a href="https://colab.research.google.com/github/maxmatical/ml-cheatsheet/blob/master/Pytorch_Lightning_BERT_Huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inspirations
https://curiousily.com/posts/multi-label-text-classification-with-bert-and-pytorch-lightning/

https://github.com/mgrankin/over9000/blob/master/train.py


https://medium.com/pytorch/getting-started-with-ray-lightning-easy-multi-node-pytorch-lightning-training-e639031aff8b


In [ ]:
!nvidia-smi

Sun Nov 28 00:10:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install transformers
!pip install pytorch_lightning
!pip install torchmetrics
!pip install torch-ema

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import OneCycleLR # , ReduceLROnPlateau
from torch.optim import AdamW

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
  
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor, StochasticWeightAveraging
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

In [ ]:
# pretrained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
model = AutoModel.from_pretrained("distilroberta-base")

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Data

In [ ]:
!gdown --id 1VuQ-U7TtggShMeuRSA_hzC8qGDl2LRkr

Downloading...
From: https://drive.google.com/uc?id=1VuQ-U7TtggShMeuRSA_hzC8qGDl2LRkr
To: /content/toxic_comments.csv
100% 68.8M/68.8M [00:00<00:00, 122MB/s] 


In [ ]:
df = pd.read_csv("toxic_comments.csv")

df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.15)

In [ ]:
# subsample clean comments
LABEL_COLUMNS = df.columns.tolist()[2:]

train_toxic = train_df[train_df[LABEL_COLUMNS].sum(axis=1) > 0]
train_clean = train_df[train_df[LABEL_COLUMNS].sum(axis=1) == 0]

train_df = pd.concat([
  train_toxic,
  train_clean.sample(15_000)
])

train_df.shape, val_df.shape

((28769, 8), (23936, 8))

In [ ]:
# take only a subsample of each train_df and val_df for faster iterations
train_df = train_df.sample(10000)
val_df = val_df.sample(10000)

train_df.shape, val_df.shape

((10000, 8), (10000, 8))

## Creating Dataset and Lightning Data Module

In [ ]:
# set batch size and max seq_len
bs = 12
seq_len = 256

In [ ]:
class ToxicCommentsDataset(Dataset):

  def __init__(
    self,
    data: pd.DataFrame,
    tokenizer: AutoTokenizer,
    max_token_len: int = 128

  ):
    self.tokenizer = tokenizer
    self.data = data
    self.max_token_len = max_token_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]
    comment_text = data_row.comment_text
    labels = data_row[LABEL_COLUMNS]

    encoding = self.tokenizer.encode_plus(
      comment_text,
      add_special_tokens=True,
      max_length=self.max_token_len,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',

    )

    return dict(
      comment_text=comment_text,
      input_ids=encoding["input_ids"].flatten(),
      attention_mask=encoding["attention_mask"].flatten(),
      labels = torch.IntTensor(labels)
    #   labels=torch.FloatTensor(labels)
    )

In [ ]:
# test
train_dataset = ToxicCommentsDataset(
  train_df,
  tokenizer,
  max_token_len=seq_len
)

sample_item = train_dataset[0]
sample_item.keys()

dict_keys(['comment_text', 'input_ids', 'attention_mask', 'labels'])

In [ ]:
print(sample_item["comment_text"], sample_item["labels"])
print(sample_item["input_ids"].shape)

"

Tell you the truth, I really don't mind this block at all. I did harass. That is what you do in a war where no one plays by the rules. Quite frankly I am fed up with being constantly blocked by silly Wiki morons and am glad I said it. We all know the politics around here so let's stop talking past each other, right? Look at why I was deleted leading to all this, not after World war III was in mid session. Who Started it? Not I! Until Wikipedia deals with morons I have no choice but to retain this position as this madness known as ""Wikipedia"" writes trash on my life. Liberals will do what liberals do: Level the playing field for the idiots, and I will be there to respond." tensor([1, 0, 0, 0, 0, 0], dtype=torch.int32)
torch.Size([256])


In [ ]:
class ToxicCommentsDataModule(pl.LightningDataModule):
  def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_token_len=512):
    super().__init__()
    self.train_df, self.test_df = train_df, test_df
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.max_token_len = max_token_len

  def setup(self, stage=None):
    self.train_dataset = ToxicCommentsDataset(
        self.train_df,
        self.tokenizer,
        self.max_token_len
    )

    self.test_dataset = ToxicCommentsDataset(
        self.test_df,
        self.tokenizer,
        self.max_token_len
    )

  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size = self.batch_size,
        shuffle = True,
        num_workers=1
    )

  def val_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size = self.batch_size,
        shuffle = False,
        num_workers=1
    )

  def test_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size = self.batch_size,
        shuffle = False,
        num_workers=1
    )

In [ ]:
data_module = ToxicCommentsDataModule(
    train_df,
    val_df,
    tokenizer,
    batch_size = bs,
    max_token_len = seq_len
)


In [ ]:
data_module.setup() # call this before getting len of dataloader

In [ ]:
len(data_module.train_dataloader())

834

# Model

In [ ]:
sample_batch = next(iter(DataLoader(train_dataset, batch_size=8, num_workers=1)))
sample_batch["input_ids"].shape, sample_batch["attention_mask"].shape

(torch.Size([8, 256]), torch.Size([8, 256]))

In [ ]:
output = model(sample_batch["input_ids"], sample_batch["attention_mask"])

In [ ]:
output.pooler_output.shape

torch.Size([8, 768])

In [ ]:
# test lr schedule with https://www.kaggle.com/isbhargav/guide-to-pytorch-learning-rate-scheduling
# test wtih flat cos lr

In [ ]:
class BertModel(nn.Module):
  def __init__(self, n_classes: int):
    super().__init__()
    self.model = model
    self.classifier = nn.Linear(self.model.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask): 
    out = self.model(input_ids, attention_mask=attention_mask)
    out = self.classifier(out.pooler_output)
    return out

In [ ]:
bert_model = BertModel(len(LABEL_COLUMNS))

In [ ]:
# bert_model(sample_batch["input_ids"], sample_batch["attention_mask"]).shape  # should be bs x 6

In [ ]:
class ToxicCommentClassifier(pl.LightningModule):
  def __init__(self, pytorch_model: nn.Module, total_steps: int, lr: float = 2e-5):
    super().__init__()
    self.lr = lr
    self.total_steps = total_steps # for lr schedule

    self.pytorch_model = pytorch_model
    self.criterion = nn.BCEWithLogitsLoss() # bce for multi-label
    # self.criterion = nn.BCELoss()

    # metrics 
    self.accuracy = torchmetrics.Accuracy()

    # use automatic optimization
    self.automatic_optimization = True

  def forward(self, input_ids, attention_mask):
    """
    forward step for lightning module
    """
    out = self.pytorch_model(input_ids, attention_mask=attention_mask)
    return out

  def training_step(self, batch, batch_idx):
    """
    using automatic optimization: no need to define opt steps etc.
    """
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    out = self(input_ids, attention_mask)
       
    # compute loss + metrics
    loss = self.criterion(out, labels.to(dtype=torch.float32)) # cast to float because labels is in Int
    acc = self.accuracy(out, labels)

    # log loss and metrics each k steps and each epoch
    log_values = {"train_loss": loss, "train_acc": acc}
    self.log_dict(log_values, sync_dist=True, prog_bar=True, on_step=True, on_epoch=True)
    return {"loss": loss, "accuracy": acc, "predictions": out, "labels": labels}

  def validation_step(self, batch, batch_idx):
    """
    validation step
    get validation loss and accuracy metrics
    """
    # batch comes from dataset
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    out = self(input_ids, attention_mask)

    # compute loss + metrics
    loss = self.criterion(out, labels.to(dtype=torch.float32)) # cast to float because labels is in Int
    acc = self.accuracy(out, labels)
    # log loss and metrics each k steps and each epoch
    log_values = {"val_loss": loss, "val_acc": acc}
    self.log_dict(log_values, sync_dist=True, prog_bar=True, on_step=True, on_epoch=True)
    return {"val_loss": loss, "val_accuracy": acc}

  def training_epoch_end(self, outputs):
    """
    after every training epoch, log train metrics

    note: this might not be needed if logging is done for each step and epoch
    """
    # log validation loss and metrics
    avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
    avg_acc = torch.stack([x["accuracy"] for x in outputs]).mean()

    self.log("avg_train_loss", avg_loss, sync_dist=True, prog_bar=True)
    self.log("train_accuracy", avg_acc, sync_dist=True, prog_bar=True)

  def validation_epoch_end(self, outputs):
    """
    after every training epoch, log val metrics

    note: this might not be needed if logging is done for each step and epoch
    """
    # log validation loss and metrics
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    avg_acc = torch.stack([x["val_accuracy"] for x in outputs]).mean()
    self.log("avg_val_loss", avg_loss, sync_dist=True, prog_bar=True)
    self.log("avg_val_accuracy", avg_acc, sync_dist=True, prog_bar=True)

    # reduce lr on plateau scheduler
    # one_cylce_sch, reduce_lr_on_plateau_sch = self.lr_schedulers()
    # sch.step(self.trainer.callback_metrics["val_accuracy"])

  def configure_optimizers(self):
    # TODO: 
    # 1. automatically find steps for onecycleLR
    # 2. add flat cos
    # 3. make lr scheduler configurable(one_cycle vs flat_cos)

    optimizer = AdamW(self.parameters(), lr=self.lr)

    scheduler = OneCycleLR(
      optimizer,
      max_lr=self.lr,
      pct_start=0.3,
      total_steps=self.total_steps
    )
    return [optimizer], [scheduler]

In [ ]:
# training epoch related hyperparams
steps_per_epoch = len(data_module.train_dataloader())
n_epochs = 5

total_steps = steps_per_epoch * n_epochs
print(f"total_steps: {total_steps}")

total_steps: 4170


Note: if using multiple gpus, total steps calculation is something like
```
steps_per_epoch = len(data_module.train_dataloader())

# for gpu or tpu cores
num_devices = 4
if tpu_cores:
    num_devices = max(num_devices, tpu_cores)

accumulate_grad_batches = 1 # for no gradient accumulation

effective_accum = accumulate_grad_batches * num_devices
total_steps (steps_per_epoch // effective_accum) * n_epochs

```

In [ ]:
toxic_comment_model = ToxicCommentClassifier(bert_model, total_steps = total_steps)

In [ ]:
# toxic_comment_model(sample_batch["input_ids"], sample_batch["attention_mask"])[1].shape  # should be bs x 6

# Trainer

defining the trainer and running `trainer.fit` 

In [ ]:
# callbacks
lr_monitor_cb = LearningRateMonitor(logging_interval='step')

checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="avg_val_accuracy",
  mode="max"
)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=6)

# swa = StochasticWeightAveraging(swa_epoch_start=0.8, swa_lrs=None)

logger = TensorBoardLogger("lightning_logs", name="toxic-comments")

In [ ]:
# standard callback, checkpoint and early stopping
# callbacks = [lr_monitor_cb, checkpoint_callback, early_stopping_callback]

# only log lr 
# not using SWA because switches out one cycle lr for SWALR scheduler
callbacks = [lr_monitor_cb]

In [ ]:
trainer = pl.Trainer(
    logger=logger,
    callbacks = callbacks,
    precision=16,
    max_epochs = n_epochs,
    accelerator="gpu",
    devices=1,
)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
# optional: currently not working?
# lr_finder = trainer.tuner.lr_find(model=toxic_comment_model, datamodule=data_module)

In [ ]:
trainer.fit(toxic_comment_model, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


MisconfigurationException: ignored

In [ ]:
trainer.logged_metrics

{'train_accuracy': 0.9270631074905396,
 'train_loss': 0.17185573279857635,
 'val_accuracy': 0.9817500114440918,
 'val_loss': 0.05154530331492424}

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/


# Saving only the pytorch model

In [ ]:
# saving model weights
saved_model_pth = "saved_model.pth"

torch.save(toxic_comment_model.pytorch_model.state_dict(), saved_model_pth)

# Exporting model as pure pytorch 

In [ ]:
new_bert_model = BertModel(6)

new_bert_model.load_state_dict(torch.load(saved_model_pth))

<All keys matched successfully>

In [ ]:
# check values are the same

inf_preds = new_bert_model(sample_batch["input_ids"], sample_batch["attention_mask"])
print(inf_preds.shape) # should be 8 x 6
inf_preds

torch.Size([8, 6])


tensor([[ 3.3002, -2.6973,  2.7221, -5.2559,  0.7052, -3.9604],
        [ 4.4156, -0.5025,  3.8863, -3.7945,  2.5557, -2.4770],
        [ 3.4849, -2.9588,  0.6141, -4.2794,  1.5000, -1.5067],
        [ 1.9291, -5.8827, -1.8313, -5.9951, -1.6118, -4.0301],
        [ 0.1542, -5.0230, -2.0128, -4.9881, -2.5206, -4.2967],
        [ 2.1084, -3.1527, -2.2322,  0.3907, -2.0325, -3.0269],
        [ 0.9454, -6.3798, -3.6319, -5.6751, -1.6457, -4.8039],
        [ 1.1208, -5.3446, -0.0648, -5.9639, -2.0950, -5.3173]],
       grad_fn=<AddmmBackward0>)

In [ ]:
toxic_comment_model(sample_batch["input_ids"], sample_batch["attention_mask"])[1]